In [3]:
import os
from glob import glob
import itertools
import fnmatch
import random
from glob import glob
import matplotlib.pylab as plt
import seaborn as sns
import cv2
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.applications import VGG16
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
#%matplotlib inline
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import preprocess_input

In [42]:
imagePatches = glob('Cancer/**/*.png', recursive=True)
for filename in imagePatches[0:10]:
    print(filename)


Cancer\test\0\14153_idx5_x101_y2851_class0.png
Cancer\test\0\14153_idx5_x1_y2851_class0.png
Cancer\test\0\14153_idx5_x1_y2901_class0.png
Cancer\test\0\14153_idx5_x251_y2851_class0.png
Cancer\test\0\14153_idx5_x251_y2901_class0.png
Cancer\test\0\14153_idx5_x301_y601_class0.png
Cancer\test\0\14153_idx5_x301_y651_class0.png
Cancer\test\0\14153_idx5_x301_y701_class0.png
Cancer\test\0\14153_idx5_x51_y2851_class0.png
Cancer\test\0\14153_idx5_x51_y2901_class0.png


In [56]:
imagePatches[:10]

['Cancer\\test\\0\\14153_idx5_x101_y2851_class0.png',
 'Cancer\\test\\0\\14153_idx5_x1_y2851_class0.png',
 'Cancer\\test\\0\\14153_idx5_x1_y2901_class0.png',
 'Cancer\\test\\0\\14153_idx5_x251_y2851_class0.png',
 'Cancer\\test\\0\\14153_idx5_x251_y2901_class0.png',
 'Cancer\\test\\0\\14153_idx5_x301_y601_class0.png',
 'Cancer\\test\\0\\14153_idx5_x301_y651_class0.png',
 'Cancer\\test\\0\\14153_idx5_x301_y701_class0.png',
 'Cancer\\test\\0\\14153_idx5_x51_y2851_class0.png',
 'Cancer\\test\\0\\14153_idx5_x51_y2901_class0.png']

In [57]:
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(imagePatches, patternZero)
classOne = fnmatch.filter(imagePatches, patternOne)
print("IDC(-)\n\n",classZero[0:5],'\n')
print("IDC(+)\n\n",classOne[0:5])

IDC(-)

 ['Cancer\\test\\0\\14153_idx5_x101_y2851_class0.png', 'Cancer\\test\\0\\14153_idx5_x1_y2851_class0.png', 'Cancer\\test\\0\\14153_idx5_x1_y2901_class0.png', 'Cancer\\test\\0\\14153_idx5_x251_y2851_class0.png', 'Cancer\\test\\0\\14153_idx5_x251_y2901_class0.png'] 

IDC(+)

 ['Cancer\\test\\1\\14154_idx5_x1001_y651_class1.png', 'Cancer\\test\\1\\14154_idx5_x1051_y701_class1.png', 'Cancer\\test\\1\\14154_idx5_x1051_y751_class1.png', 'Cancer\\test\\1\\14154_idx5_x1051_y801_class1.png', 'Cancer\\test\\1\\14154_idx5_x1051_y851_class1.png']


In [58]:
def proc_images(lowerIndex,upperIndex):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """ 
    x = []
    y = []
    WIDTH = 50
    HEIGHT = 50
    for img in imagePatches[lowerIndex:upperIndex]:
        full_size_image = cv2.imread(img)
        x.append(cv2.resize(full_size_image, (WIDTH,HEIGHT), interpolation=cv2.INTER_CUBIC))
        if img in classZero:
            y.append(0)
        elif img in classOne:
            y.append(1)
        else:
            return
    return x,y


In [59]:
X,Y = proc_images(0,300000)

In [60]:
def describeData(a,b):
    print('Total number of images: {}'.format(len(a)))
    print('Number of IDC(-) Images: {}'.format(np.sum(b==0)))
    print('Number of IDC(+) Images: {}'.format(np.sum(b==1)))
    print('Percentage of positive images: {:.2f}%'.format(100*np.mean(b)))
    print('Image shape (Width, Height, Channels): {}'.format(a[0].shape))
describeData(X,Y)

Total number of images: 80
Number of IDC(-) Images: 0
Number of IDC(+) Images: 0
Percentage of positive images: 50.00%
Image shape (Width, Height, Channels): (50, 50, 3)


In [80]:
import numpy as np
X=np.array(X)
Y=np.array(Y)
X=X/255.0
num_classes = 2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)



from keras.utils.np_utils import to_categorical
Y_trainHot = to_categorical(Y_train, num_classes = 2)
Y_testHot = to_categorical(Y_test, num_classes = 2)




In [81]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [82]:
IMAGE_SIZE = [50,50]
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [83]:
for layer in vgg16.layers:
    layer.trainable = False
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 50, 50, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 50, 50, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 50, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 25, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 25, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 25, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [84]:
#Now, let us use features from convolutional network for RF
model=Model(inputs=vgg16.input,outputs=vgg16.get_layer('block4_pool').output)

feature_extractor=model.predict(X_train)
print(feature_extractor.shape)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)
np.save("train_features_b4p.npy",features)
print(features.shape)

X_for_RF = features #This is our X input to RF



(56, 3, 3, 512)
(56, 4608)


In [101]:
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

# Train the model on training data
RF_model.fit(X_for_RF, Y_train) #For sklearn no one hot encoding

#Send test data through same feature extractor process
X_test_feature = model.predict(X_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)
prediction_RF = RF_model.predict(X_test_features)

In [107]:
prediction_RF=prediction_RF.reshape(prediction_RF.shape[0],-1)
print(prediction_RF.shape)
Y_test=Y_test.reshape(Y_test.shape[0],-1)
print(Y_test.shape)

(24, 1)
(24, 1)


In [106]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(RF_model, prediction_RF, Y_test, cv=10, scoring ='accuracy').mean())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y wa

0.8666666666666666


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y wa